In [ ]:
%pip install python-dotenv matplotlib pandas plotly prettytable


In [ ]:
# We're going to use the requests library for http requests, 
# dotenv to load environment variables, os to get those variables, 
# and pandas to create dataframes.

import requests 
from dotenv import load_dotenv
from os import environ 
import pandas as pd

In [ ]:
load_dotenv()

# The variables below are present in the API information in Rapid.
X_RapidAPI_Key  = environ.get('X_RapidAPI_Key')
X_RapidAPI_Host = environ.get('X-RapidAPI-Host') 


# Authentication Provider Variables
# These variables are acquired from the authentication provider from when, where will you go
# create a client. To do so, just follow the step-by-step instructions in Rapid's tutorials.
client_id       = environ.get('client_id')
client_secret   = environ.get('client_secret')
username        = environ.get('username')
password        = environ.get('password')
url_token       = 'https://provider.quant1.com.br/oauth/token'

asset_name = 'AZUL4'
# API general variable.
url = "https://quantum-q1-api.p.rapidapi.com"

In [ ]:
# In this request we capture the token that we will use to authenticate our API.
response = requests.post(url_token, auth=(client_id, client_secret), data={
    'username': username, 
    'password': password,
    'scope': 'profile',
    'response_types': 'code',
    'grant_type':'password'

}).json()

In [ ]:
# This header will be used to validate the API, it will use the Rapid API key and the API host along with the oauth Access Token that was generated earlier
headers = {
	"X-RapidAPI-Key": X_RapidAPI_Key,
	"X-RapidAPI-Host": X_RapidAPI_Host,
    "Authorization": "Bearer " + response.get('access_token')
}

In [ ]:
# Function responsible for making the request in the API
def asset_all():
    response = requests.get(url+"/assets/all", headers=headers)
    return response.json()
data_all = data = asset_all()

In [ ]:
# Here we filter the assets that are present in São Paulo, Belo Horizonte and Rio de Janeiro.
# We also organize the data in ascending order according to marketcap for better visualization.
# And we create a dataframe for using the data that was treated.

data = list(filter(lambda key: key['head_office'] == 'São Paulo' or key['head_office'] == 'Belo Horizonte' or key['head_office'] == 'Rio de Janeiro', data))
data = sorted(data, key=lambda x: x['marketcap'])
data_all = sorted(data_all, key=lambda x: x['marketcap'])

data = pd.DataFrame.from_dict(data)
data_all = pd.DataFrame.from_dict(data_all)



In [ ]:
# Here we generate a scatter chart to demonstrate this relationship and to make a comparison possible.
import plotly.express as px
fig = px.scatter(data_frame=data, x='marketcap', y='employees', color='head_office')
fig.update_layout(
    title='Relationship between market value and employees in assets in São Paulo, Belo Horizonte and Rio de Janeiro.',
    xaxis_title='Marketcap',
    yaxis_title='Employees',
    template = 'plotly_white',
)
fig.write_html('first_figure.html', auto_open=True)

In [ ]:
fig = px.scatter(data_frame=data_all, x='marketcap', y='employees', color='head_office')
fig.update_layout(
    title='Relationship between market value and employees in the assets.',
    xaxis_title='Marketcap',
    yaxis_title='Employees',
    template = 'plotly_white',
)
fig.write_html('first_figure.html', auto_open=True)

In [ ]:
# Now i will show a prompt to fetch assets by name.
def asset_by_name(asset):
    response = requests.get(url+f"/assets/by/name/{asset}", headers=headers)
    return response.json()
data = asset_by_name('AZUL4')

In [ ]:
# An example of an asset data visualization
from prettytable import PrettyTable

table = PrettyTable()
field_names = ['ceo', 'cnpj', 'company_url', 'description', 'employees', 'foundation', 'head_office', 'id', 'marketcap', 'marketplace', 'sector', 'symbol']
table.field_names = field_names
table.add_row([d for d in data.values()])
print(table)
